In [1]:
import pickle
import random
import time
import os

import pandas as pd
import numpy as np
from tqdm import tqdm

import torch
from matplotlib import pyplot as plt
import soundfile as sf
import librosa
import librosa.display

tqdm.pandas()

import paule
import util
from util import (speak, normalize_cp, inv_normalize_cp, normalize_mel_librosa, inv_normalize_mel_librosa, stereo_to_mono, librosa_melspec, pad_same_to_even_seq_length, RMSELoss, mel_to_sig, pad_batch_online)
from models import *
from matplotlib import cm
from matplotlib.lines import Line2D

DIR = os.getcwd()
DEVICE = torch.device('cpu')

/Users/paule/anaconda3/envs/bonn17/lib/python3.8/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
DATA_DIR = "/Volumes/T7/Common_Voice/data/"
RESULT_DICT = "/Volumes/T7/Common_Voice/common_voice_test_planning"
SAVE_DICT = "/Volumes/T7/Common_Voice/common_voice_test_planning/results_acoustic_lr_001_lr_model_00001_5_outer_25_inner_3_batches_10_epochs_only_new"

# Initialize Paule

In [16]:
paule_model = paule.Paule(device=DEVICE)

# Prepare Test Data

In [13]:
test_data = pd.read_pickle(DATA_DIR + "common_voice_geco_words_test_subset_slim_prot4.pkl")

## Create Flac Files 

In [ ]:
"""
for i,row in test_data.iterrows():
    sr = 48000
    sig = row.wav_rec
    file_name = row.file_name
    label = row.label
    
    path = RESULT_DICT + label + "/"
    if os.path.isdir(path):
        file = path +  file_name + "_" + label + ".flac"
        if file.is_file():
            j = 1
            while file.is_file(): 
                file = path +  file_name + "_" + label + "%d.flac" % j
                j+=1
        sf.write(path +  file_name + "_" + label + "3.flac" ,sig, sr)
    else:
        os.mkdir(path)
        sf.write(path +  file_name + "_" + label + ".flac" ,sig, sr)
"""

## Random Shuffle Files

In [17]:
files = []
for dic in os.listdir(RESULT_DICT):
    if (".DS_Store" in dic) or ("results" in dic) : # .DS_Store stores custom attributes of its containing folder in macOS
        continue
    else:
        if not os.path.isdir(os.path.join(SAVE_DICT, dic)):
            os.mkdir(os.path.join(SAVE_DICT, dic))
        path = os.path.join(RESULT_DICT, dic)
        for file in os.listdir(path):
            if (".DS_Store" in file) or ("._" in file):
                continue
            else:
                file = os.path.join(path,file)
                files.append(file)
random.seed(30112021)
random.shuffle(files)

### Omit already planned files

In [18]:
already_planned_files = []
for dic in os.listdir(SAVE_DICT):
    if ".DS_Store" in dic:
        continue
    else:
        path = os.path.join(SAVE_DICT,dic)
        if os.path.isdir(path):
            for file in os.listdir(path):
                if "planned" in file: 
                    planned_file = "_".join(file.split("_")[:-1]) + ".flac" 
                    already_planned_files.append(planned_file)

unplanned_files = [file for file in files if file.split("/")[-1] not in already_planned_files]

### Load predictive model already used and further learned during planning 

In [21]:
# load model
pred_model = torch.load("/Volumes/T7/Common_Voice/common_voice_test_planning/results_acoustic_lr_001_lr_model_00001_5_outer_25_inner_3_batches_10_epochs_only_new/schwierig/common_voice_de_21967968_schwierig_pred_model.pt", map_location=DEVICE)
optimizer = torch.load("/Volumes/T7/Common_Voice/common_voice_test_planning/results_acoustic_lr_001_lr_model_00001_5_outer_25_inner_3_batches_10_epochs_only_new/schwierig/common_voice_de_21967968_schwierig_pred_optimizer.pt", map_location=DEVICE)

In [23]:
paule_model.pred_model = pred_model
paule_model.pred_optimizer = optimizer

In [24]:
len(unplanned_files)

117

# Start Planning and store results 

In [ ]:
for i, file in enumerate(unplanned_files):
    i += 109
    target_acoustic = file #path + "/" + file
    save_file = f"{SAVE_DICT}/{os.path.dirname(file).split('/')[-1]}/{os.path.basename(file)[:-5]}"
    
    results = paule_model.plan_resynth(learning_rate_planning=0.01,
            learning_rate_learning=0.001,
            target_acoustic=target_acoustic,
            initialize_from="acoustic",
            objective="acoustic",
            n_outer=5, n_inner=25,
            continue_learning=True,
            add_training_data=False,
            log_ii=1,
            log_semantics=True,
            n_batches=3, batch_size=8, n_epochs=10,
            log_gradients=False,
            plot=True, plot_save_file=save_file, 
            seed=None,
            verbose=True)
    
    # save model and optimizer
    torch.save(paule_model.pred_model, f"{save_file}_{str(i)}_pred_model.pt")
    torch.save(paule_model.pred_optimizer, f"{save_file}_{str(i)}_pred_optimizer.pt")

    # save results without model and optimizer
    with open(f"{save_file}.pkl", 'wb') as pfile:
        pickle.dump(results, pfile)


    # save initial and planned flac
    prod_sr = results.prod_sr
    sig_initial = results.sig_steps[0]
    sf.write(save_file + "_initial.flac", sig_initial, prod_sr)
    prod_sig = results.prod_sig
    sf.write(save_file + "_planned.flac", prod_sig, prod_sr)

    # save svgs
    planned_cp = results.planned_cp
    path = save_file + '_svgs/'
    if not os.path.exists(path):
        os.mkdir(path)
    util.export_svgs(util.inv_normalize_cp(planned_cp), path=path)

    # ffmpeg -r 80 -width 600 -i tract%05d.svg -i planned_0.flac planned_0.mp4
    # /usr/bin/ffmpeg -r 80 -width 600 -i /home/tino/Documents/phd/projects/paule/results/000003-Wissenschaft_svgs/tract%05d.svg -i results/000003-Wissenschaft_planned.flac planned.mp4


    # save loss plot
    fig, ax = plt.subplots(figsize=(15, 8), facecolor="white")
    ax.plot(results.planned_loss_steps, label="planned loss", c="C0")
    ax.legend()
    fig.savefig(f"{save_file}_loss.png")

    fig, ax = plt.subplots(figsize=(15, 8), facecolor="white")
    ax.plot(results.prod_loss_steps, label="produced mel loss", c="C1")
    ax.plot(results.planned_mel_loss_steps, label="planned mel loss", c="C0")
    ax.legend()
    fig.savefig(f"{save_file}_loss_mel.png")

    # save subloss plot
    fig, ax = plt.subplots(figsize=(15, 8), facecolor="white")
    ax.plot(results.vel_loss_steps, label="vel loss", c="C2")
    ax.plot(results.jerk_loss_steps, label="jerk loss", c="C3")
    ax.legend()
    fig.savefig(f"{save_file}_loss_subloss.png")

    # save semvec loss plot
    fig, ax = plt.subplots(figsize=(15, 8), facecolor="white")
    ax.plot(results.pred_semvec_loss_steps, label="planned semvec loss", c="C0")
    ax.plot(results.prod_semvec_loss_steps, label="produced semvec loss", c="C1")
    ax.legend()
    fig.savefig(f"{save_file}_loss_semvec.png")
    
    
    # save cps
    # Vocal Tract CPs
    fig, ax = plt.subplots(facecolor="white", figsize=(15, 10))
    colors = ["C%d" % i for i in range(19)]

    for i in range(8):
        ax.plot(results.planned_cp[:, i], color=colors[i], ls = 'solid', lw = 2)
        ax.plot(results.inital_cp[:, i], color=colors[i], ls = 'dotted', lw = 4)
    
    legend_elements = [Line2D([0], [0], color='black', ls="solid", lw=2, label='Planned Cp'),#
                       Line2D([0], [0], color='black',ls ="dotted", lw = 4 , label='Inital CP')]
    
    ax.set_ylim((-1.1, 1.1))
    plt.legend(handles=legend_elements, fontsize=fontsize_params, bbox_to_anchor=(1.0, 1),frameon = False) 
    ax.tick_params(axis='both', labelsize=fontsize_params)
    ax.set_ylabel('Normalized Position' , fontsize=fontsize_y, labelpad=20)
    ax.set_xlabel('Timestep (2.5ms)' , fontsize=fontsize_x, labelpad=20)  
    plt.title("Vocal Tract Cps: '%s'" % save_file.split("/")[-1], fontsize=18, pad=10)
    fig.savefig(f"{save_file}_vocal_tract_cps.png")
    
    # Glottis CPs
    fig, ax = plt.subplots(facecolor="white", figsize=(15, 10))
    colors = ["C%d" % i for i in range(19)]

    for i in range(8):
        ax.plot(results.planned_cp[:, 19+i], color=colors[i], ls = 'solid', lw = 2)
        ax.plot(results.inital_cp[:, 19+i], color=colors[i], ls = 'dotted', lw = 4)
    
    legend_elements = [Line2D([0], [0], color='black', ls="solid", lw=2, label='Planned Cp'),#
                       Line2D([0], [0], color='black',ls ="dotted", lw = 4 , label='Inital CP')]
    
    ax.set_ylim((-1.1, 1.1))
    plt.legend(handles=legend_elements, fontsize=fontsize_params, bbox_to_anchor=(1.0, 1),frameon = False) 
    ax.tick_params(axis='both', labelsize=fontsize_params)
    ax.set_ylabel('Normalized Position' , fontsize=fontsize_y, labelpad=20)
    ax.set_xlabel('Timestep (2.5ms)' , fontsize=fontsize_x, labelpad=20)
    plt.title("Glottis Cps: '%s'" % save_file.split("/")[-1], fontsize=18, pad=10)
    fig.savefig(f"{save_file}_glottis_cps.png")


    

    ax.set_ylim((-1.1, 1.1))
    plt.legend(handles=legend_elements, fontsize=fontsize_params, bbox_to_anchor=(1.0, 1),frameon = False) 
    ax.tick_params(axis='both', labelsize=fontsize_params)
    #ax.set_xticks([])
    ax.set_ylabel('Normalized Position' , fontsize=fontsize_y, labelpad=20)
    ax.set_xlabel('Timestep (2.5ms)' , fontsize=fontsize_x, labelpad=20)
    #plt.savefig("predicted_vocal_tract_cps_%s.jpg" % label)
    #plt.show()


  0%|          | 0/5 [00:00<?, ?it/s]

Iteration 0
Planned Loss:  0.22908913541769396
Mel Loss:  0.17816797927069947
Vel Loss:  0.03845465304127829
Jerk Loss:  0.0124665031057162
Semvec Loss:  0.019167911214873077
Produced Mel Loss:  0.18612341082838402
Produced Semvec Loss:  0.019719960080490737

Iteration 1
Planned Loss:  0.251723237215572
Mel Loss:  0.16968208129802548
Vel Loss:  0.040823305236616286
Jerk Loss:  0.0412178506809302
Semvec Loss:  0.018795617858043186
Produced Mel Loss:  0.1822402418160016
Produced Semvec Loss:  0.0234837045058742

Iteration 2
Planned Loss:  0.23606070125585388
Mel Loss:  0.1644582278687264
Vel Loss:  0.03997654417692707
Jerk Loss:  0.0316259292102004
Semvec Loss:  0.019055366520360897
Produced Mel Loss:  0.1788661799182711
Produced Semvec Loss:  0.02385420024797145

Iteration 3
Planned Loss:  0.22946096410473227
Mel Loss:  0.1609669415878996
Vel Loss:  0.03925904171754677
Jerk Loss:  0.029234980799285923
Semvec Loss:  0.01959362261748636
Produced Mel Loss:  0.17277737976395713
Produced Sem

/Users/paule/anaconda3/envs/bonn17/lib/python3.8/site-packages/librosa/display.py:974: MatplotlibDeprecationWarning: The 'basey' parameter of __init__() has been renamed 'base' since Matplotlib 3.3; support for the old name will be dropped two minor releases later.
  scaler(mode, **kwargs)
/Users/paule/anaconda3/envs/bonn17/lib/python3.8/site-packages/librosa/display.py:974: MatplotlibDeprecationWarning: The 'linthreshy' parameter of __init__() has been renamed 'linthresh' since Matplotlib 3.3; support for the old name will be dropped two minor releases later.
  scaler(mode, **kwargs)
 20%|██        | 1/5 [02:07<08:28, 127.00s/it]

Iteration 0
Planned Loss:  0.1796940329840428
Mel Loss:  0.13640448745200331
Vel Loss:  0.03614723376720011
Jerk Loss:  0.0071423117648393665
Semvec Loss:  0.01761332793455516
Produced Mel Loss:  0.14403823421119216
Produced Semvec Loss:  0.015838105979678562

Iteration 1
Planned Loss:  0.17881654364109675
Mel Loss:  0.1361069814070399
Vel Loss:  0.036038574357258865
Jerk Loss:  0.006670987876798011
Semvec Loss:  0.017721356834713778
Produced Mel Loss:  0.14420290576360884
Produced Semvec Loss:  0.016304405223256595

Iteration 2
Planned Loss:  0.17834822095302624
Mel Loss:  0.1358296908356364
Vel Loss:  0.03593942417088244
Jerk Loss:  0.006579105946507411
Semvec Loss:  0.017791441089973555
Produced Mel Loss:  0.1444326484376707
Produced Semvec Loss:  0.01702677276533132

Iteration 3
Planned Loss:  0.17775178233649488
Mel Loss:  0.13557797308439934
Vel Loss:  0.03583731739577273
Jerk Loss:  0.0063364918563228265
Semvec Loss:  0.017827294619452514
Produced Mel Loss:  0.1436050395493957
P

 40%|████      | 2/5 [04:15<06:22, 127.45s/it]

...

Iteration 0
Planned Loss:  0.16585840371468774
Mel Loss:  0.1281207135864776
Vel Loss:  0.0297893168732392
Jerk Loss:  0.007948373254970949
Semvec Loss:  0.014174945371205479
Produced Mel Loss:  0.17414808641606305
Produced Semvec Loss:  0.017864893688634326

Iteration 1
Planned Loss:  0.16482328297226487
Mel Loss:  0.1275145557620809
Vel Loss:  0.02968774063494835
Jerk Loss:  0.007620986575235598
Semvec Loss:  0.013874597069846535
Produced Mel Loss:  0.17466475495046618
Produced Semvec Loss:  0.016663623975133143

Iteration 2
Planned Loss:  0.16424462072666782
Mel Loss:  0.12708996365272324
Vel Loss:  0.029597811251159803
Jerk Loss:  0.007556845822784798
Semvec Loss:  0.013666800943294262
Produced Mel Loss:  0.17458724193951505
Produced Semvec Loss:  0.017383278376945793

Iteration 3
Planned Loss:  0.16329005070924435
Mel Loss:  0.1266167880949631
Vel Loss:  0.029500899340729105
Jerk Loss:  0.007172363273552136
Semvec Loss:  0.013341900887949494
Produced Mel Loss:  0.17486062060598495

 40%|████      | 2/5 [07:31<11:17, 225.89s/it]

Iteration 0
Planned Loss:  0.1525415967513209
Mel Loss:  0.1206449824441501
Vel Loss:  0.027574403442800682
Jerk Loss:  0.004322210864370124
Semvec Loss:  0.011337005107798363
Produced Mel Loss:  0.1768401509381087
Produced Semvec Loss:  0.01856189726923448

Iteration 1
Planned Loss:  0.15237017989688273
Mel Loss:  0.120449319861548
Vel Loss:  0.027501376842274106
Jerk Loss:  0.0044194831930606145
Semvec Loss:  0.011298927205667237
Produced Mel Loss:  0.17841894963999205
Produced Semvec Loss:  0.01797842347028136

Iteration 2
Planned Loss:  0.15181615347302627
Mel Loss:  0.12025448487708201
Vel Loss:  0.027422827112681285
Jerk Loss:  0.004138841483262969
Semvec Loss:  0.011242072068892366
Produced Mel Loss:  0.17730232380867803
Produced Semvec Loss:  0.018229630818948527

Iteration 3
Planned Loss:  0.15183511780544137
Mel Loss:  0.12007729029805088
Vel Loss:  0.02735665145002144
Jerk Loss:  0.004401176057369074
Semvec Loss:  0.011195108221186956
Produced Mel Loss:  0.1805994514131516
P

 60%|██████    | 3/5 [11:19<07:32, 226.40s/it]

Iteration 0
Planned Loss:  0.14677115658050574
Mel Loss:  0.11640829368758662
Vel Loss:  0.026040285328465182
Jerk Loss:  0.004322577564453948
Semvec Loss:  0.010646309753760526
Produced Mel Loss:  0.19887563307667294
Produced Semvec Loss:  0.029241362927654477

Iteration 1
Planned Loss:  0.1454124023657392
Mel Loss:  0.11626049271912285
Vel Loss:  0.02597179938730109
Jerk Loss:  0.0031801102593152913
Semvec Loss:  0.010609968807926185
Produced Mel Loss:  0.19913152958285452
Produced Semvec Loss:  0.02937580734293617

Iteration 2
Planned Loss:  0.1464817267485588
Mel Loss:  0.11622148337041156
Vel Loss:  0.02593916245442891
Jerk Loss:  0.0043210809237183324
Semvec Loss:  0.010722307776562844
Produced Mel Loss:  0.19927822311805085
Produced Semvec Loss:  0.029801280790037497

Iteration 3
Planned Loss:  0.14504228041397427
Mel Loss:  0.1160424770882258
Vel Loss:  0.02586898960364209
Jerk Loss:  0.0031308137221063817
Semvec Loss:  0.010680726072765833
Produced Mel Loss:  0.202883475729784

## Store final losses to txt 

In [8]:
file_names = []
with open(SAVE_DICT + '/results_loss.txt', 'w') as txt:
    for dic in os.listdir(SAVE_DICT):
        if ".DS_Store" in dic:
            continue
        else:
            path = os.path.join(SAVE_DICT,dic)
            if os.path.isdir(path):
                for file in os.listdir(path):
                    if ".pkl" in file: 
                        with open(os.path.join(path,file), 'rb') as f:
                            results = pickle.load(f)
                            file_name = "_".join(file.split("_")[:-1])
                            prod_mel_loss = results.prod_loss_steps[-1]
                            pred_mel_loss = results.planned_mel_loss_steps[-1]
                            vel_loss = results.vel_loss_steps[-1]
                            jerk_loss = results.jerk_loss_steps[-1]
                        file_names.append(file_name)    
                        txt.write(file_name + ": %.5f %.5f %.5f %.5f\n" % (prod_mel_loss, pred_mel_loss, vel_loss, jerk_loss))

# Plot Model Loss for continued learning 

In [ ]:
model_loss = []
with open(SAVE_DICT + '/results_loss.txt', 'w') as txt:
    for dic in os.listdir(SAVE_DICT):
        if ".DS_Store" in dic:
            continue
        else:
            path = os.path.join(SAVE_DICT,dic)
            if os.path.isdir(path):
                for file in os.listdir(path):
                    if ".pkl" in file: 
                        with open(os.path.join(path,file), 'rb') as f:
                            results = pickle.load(f)
                            model_loss += list(results.model_loss)#results[21]

In [ ]:
fig, ax = plt.subplots(figsize = (15,8), facecolor = "white")
ax.plot(model_loss, label = "model loss", c = "C0")
#ax.plot(range(len(model_loss)),np.repeat(np.mean(model_loss),len(model_loss)) ,c = "C1")
ax.legend()

# Data Wrangling with Final Results

In [ ]:
final_results = pd.DataFrame(columns=['file_name','label', 'planned_cp', 'initial_cp', 'target_sig','initial_sig', 'prod_sig', 'target_mel', 
                                      'initial_mel','prod_mel','pred_mel', 'prod_mel_loss', 'pred_mel_loss', 'vel_loss', 'jerk_loss',
                                     'prod_semvec','pred_semvec','prod_semvec_loss', 'pred_semvec_loss'])
ix = 0

for dic in os.listdir(SAVE_DICT):
    if ".DS_Store" in dic:
        continue
    else:
        path = os.path.join(SAVE_DICT,dic)
        if os.path.isdir(path):
            for file in os.listdir(path):
                if ".pkl" in file: 
                    with open(os.path.join(path,file), 'rb') as f:
                        results = pickle.load(f)
                        
                        file_name = "_".join(file.split("_")[:-1])
                        label=file.split("_")[-1][:-4]
                        planned_cp = results.planned_cp
                        initial_cp = results.initial_cp
                        target_sig = results.target_sig
                        prod_sig = results.prod_sig
                        target_mel = results.target_mel
                        
                        initial_sig, sr = speak(inv_normalize_cp(initial_cp))
                        initial_mel = librosa_melspec(initial_sig, sr)
                        initial_mel = normalize_mel_librosa(initial_mel)
                        prod_mel = results.prod_mel
                        pred_mel = results.pred_mel
                        prod_mel_loss = results.prod_loss_steps[-1]
                        pred_mel_loss = results.planned_mel_loss_steps[-1]
                        vel_loss = results.vel_loss_steps[-1]
                        jerk_loss = results.jerk_loss_steps[-1]
                        prod_semvec = results.prod_semvec_steps.[-1]
                        pred_semvec = results.pred_semvec_steps[-1]
                        prod_semvec_loss = results.prod_semvec_loss_steps[-1]
                        pred_semvec_loss = results.pred_semvec_loss_steps[-1]
                        
                        
                        final_results.loc[ix] = [file_name,label,planned_cp,inv_cp,target_sig,initial_sig,prod_sig,target_mel,initial_mel,prod_mel,
                                                pred_mel,prod_mel_loss,pred_mel_loss,vel_loss,jerk_loss,prod_semvec,
                                                pred_semvec,prod_semvec_loss,pred_semvec_loss]
                        ix+=1

In [ ]:
final_results.to_pickle(SAVE_DICT+"/final_results.pkl", protocol=4)